In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime 

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import warnings
import pmdarima as pm
import random
random.seed(10)
warnings.filterwarnings("ignore")

#https://par.nsf.gov/servlets/purl/10186768#:~:text=%E2%80%93%20Compare%20the%20performance%20of%20LSTM,indicating%20the%20superiority%20of%20LSTM.

In [ ]:
passengers_df = pd.read_csv('data/AIRLINE_PASSENGERS.csv', parse_dates=['Date'])
alcohol_df = pd.read_csv('data/ALCOHOL_SALES.csv', parse_dates=['Date'])
beer_df = pd.read_csv('data/AUS_BEER_PRODUCTION.csv', parse_dates=['Date'])
electric_df = pd.read_csv('data/ELECTRIC_PRODUCTION.csv', parse_dates=['Date'])
minTemp_df = pd.read_csv('data/MIN_TEMP.csv', parse_dates=['Date'])
gdp_df = pd.read_csv('data/NOR_GDP.csv', parse_dates=['Date'])
#population_df = pd.read_csv('data/POPULATION.csv', parse_dates=['Date'])
sunspots_df = pd.read_csv('data/SUNSPOTS.csv', parse_dates=['Date'])
SP_df = pd.read_csv('data/SP.csv', parse_dates=['Date'])
yahoo_df = pd.read_csv('data/YAHOO.csv', parse_dates=['Date'])
tesla_df = pd.read_csv('data/TESLA.csv', parse_dates=['Date'])




def set_name(dfs,names):
    for ind, df in enumerate(dfs):
        df.name = names[ind]
    return dfs
name_list = ['passengers_df', 'alcohol_df', 'beer_df', 
             'electric_df', 'minTemp_df', 'gdp_df', 'sunspots_df','SP_df', 'yahoo_df', 'tesla_df']


dfs_list = []
dfs_list.append(passengers_df)
dfs_list.append(alcohol_df)
dfs_list.append(beer_df)
dfs_list.append(electric_df)
dfs_list.append(minTemp_df)
dfs_list.append(gdp_df)
dfs_list.append(sunspots_df)
dfs_list.append(SP_df)
dfs_list.append(yahoo_df)
dfs_list.append(tesla_df)




dfs_list = set_name(dfs_list, name_list)


In [ ]:
def log_transform(dfs):
    transformed = []
    lambda_list = []
    for df in dfs:
        col_transformed = boxcox(df[df.columns[1]], 0)
        col_transformed = pd.DataFrame({'Date': df[df.columns[0]], f'{df.columns[1]}': col_transformed})
        transformed.append(col_transformed)
        lambda_list.append(0) 
    transformed = set_name(transformed, name_list)
    return transformed, lambda_list

def inverse_transform(transformed_df, lambda_):
    inv_df = inv_boxcox(transformed_df, lambda_)
    return inv_df

log_transformed_list, best_lambdas = log_transform(dfs_list)


In [ ]:
def get_mape(actual, pred):
    return np.mean(np.abs((actual - pred)/actual))*100

In [ ]:
def get_m(data):
    #setting m
    days = (data[data.columns[0]][1] - data[data.columns[0]][0]).days
    if days >= 1 and days <= 4:
        m = 7
    elif days <= 31 and days >= 28:
        m = 12
    elif days == 7:
        m = 52
    return m

In [ ]:
def split_train_test(df, ratio, nr_of_forecasts):
   
     
    split_range = int(len(df)* ratio)
    train, test, forecasts = np.array(df[df.columns[1]][0:split_range]), np.array(df[df.columns[1]][split_range:len(df)-nr_of_forecasts]), np.array(df[df.columns[1]][len(df)-nr_of_forecasts:len(df)]) 
    return train, test, forecasts


In [ ]:
def get_pdq_values(df, max_p, max_q, max_P = None, max_Q = None, which_model = None):
    if which_model == "ARIMA":
        model = pm.auto_arima(df[df.columns[1]], information_criterion= 'aic', max_p = max_p, max_q = max_q,  MAX_P = 0, MAX_D= 0, MAX_Q = 0, stepwise= True )
        p = model.get_params()['order'][0]
        d = model.get_params()['order'][1]
        q = model.get_params()['order'][2]
        return p, d, q
    else:
        model = pm.auto_arima(df[df.columns[1]], information_criterion= 'aic', max_p = max_p, max_q = max_q,  max_P = max_P, max_D= 2,  max_Q = max_Q, m= get_m(df),stepwise= False )
        p = model.get_params()['order'][0]
        d = model.get_params()['order'][1]
        q = model.get_params()['order'][2]
        P = model.get_params()['seasonal_order'][0]
        D = model.get_params()['seasonal_order'][1]
        Q = model.get_params()['seasonal_order'][2]
        m = model.get_params()['seasonal_order'][3]
        return p, d, q, P, D, Q, m


In [ ]:
def get_arima_accuracy(df,best_lambda, fcs_nr_list, max_p, max_q, max_P = None, max_Q = None, which_model = None):
    
    
    training, test, all_forecasts = split_train_test(df, ratio = 0.75, nr_of_forecasts= 10)
    training_fit = training
    
    if which_model == "ARIMA":
        p, d, q = get_pdq_values(df.iloc[:-10], max_p, max_q, which_model="ARIMA")
        model = ARIMA(training, order = (p, d, q))
    else:
        p, d, q, P, D, Q, m = get_pdq_values(df.iloc[:-10], max_p, max_q, max_P, max_Q, which_model= "SARIMA")
        print(p, d, q, P, D, Q, m)
        model = SARIMAX(training, order=(p, d, q), seasonal_order=(P, D, Q, m))
    model_fit = model.fit()
        
    training_fcs = model_fit.predict(dynamic = False)
    training, training_fcs = inverse_transform(training, best_lambda), inverse_transform(training_fcs, best_lambda)
    train_rmse = (mean_squared_error(training, training_fcs))**0.5
    train_mape = get_mape(training, training_fcs)
   
    print(f'train rmse: {train_rmse}')
    print(f'train mape: {train_mape}')
    
    test_fcs = np.array([])
    for i in range(len(test)):
        if which_model == "ARIMA":
            model = ARIMA(training_fit, order = (p, d, q))
        else:
            model = SARIMAX(training_fit, order = (p, d, q), seasonal_order = (P, D, Q,m))
        model_fit = model.fit()
        single_fc = model_fit.forecast()
        
        test_fcs = np.append(test_fcs,single_fc)
        training_fit = np.append(training_fit, test[i])
        
        
      
        
   
    test, test_fcs = inverse_transform(test, best_lambda), inverse_transform(test_fcs, best_lambda)
    
    
    

    #plt.plot(training)
    #plt.plot(training_fcs)
    #plt.show()
    
    
    test_rmse = (mean_squared_error(test,test_fcs))**0.5
    test_mape = get_mape(test, test_fcs)
    #plt.plot(test)
    #plt.plot(test_fcs)
    #plt.show()
    print(f'test rmse: {test_rmse}')
    print(f'test mape: {test_mape}')
    
    forecasts_rmse_list = []
    forecasts_mape_list = []
    for fcs in fcs_nr_list:
        forecasts_predictions = model_fit.forecast(steps = fcs)
        
    
        
        
        forecasts_rmse = np.round((mean_squared_error(inverse_transform(all_forecasts, best_lambda)[0:fcs],inverse_transform(forecasts_predictions, best_lambda)))**0.5, 4)
        forecasts_mape = np.round(get_mape(inverse_transform(all_forecasts, best_lambda)[0:fcs],inverse_transform(forecasts_predictions, best_lambda)), 4)
        
        
        
        forecasts_rmse_list.append(forecasts_rmse)
        forecasts_mape_list.append(forecasts_mape)
    #plt.plot(forecasts, marker = 'o')
    #plt.plot(forecasts_predictions, marker = 'x')
    #plt.show()
    
    print(f'forecasts rmse: {forecasts_rmse_list}')
    print(f'forecasts mape: {forecasts_mape_list}')
    return np.round(train_rmse, 4), np.round(train_mape, 4), np.round(test_rmse, 4), np.round(test_mape, 4), forecasts_rmse_list, forecasts_mape_list
    
    

                  


In [ ]:
def save_cv(filepath, train_rmse, train_mape, test_rmse, test_mape, fc_rmse, fc_mape):
    
    df = pd.DataFrame({'Data': name_list, 'Train RMSE': train_rmse,'Train MAPE': train_mape,
                       'Test RMSE': test_rmse, 'Test MAPE': test_mape, '1-step RMSE': [x[0] for x in fc_rmse],
                       '3-step RMSE': [x[1] for x in fc_rmse], '5-step RMSE': [x[2] for x in fc_rmse],
                        '10-step RMSE': [x[3] for x in fc_rmse],'1-step MAPE': [x[0] for x in fc_mape],
                       '3-step MAPE': [x[1] for x in fc_mape],'5-step MAPE': [x[2] for x in fc_mape],
                       '10-step MAPE': [x[3] for x in fc_mape] })
    df.to_csv(filepath, index = False)
    return df

## ARIMA

In [ ]:
train_rmse_list, train_mape_list, test_rmse_list, test_mape_list, fc_rmse_list_list,fc_mape_list_list  = [], [], [], [], [], []
nr_of_forecasts_list = [1, 3, 5, 10]
for ind, df in enumerate(log_transformed_list):
    print(df.name)
    train_rmse, train_mape, test_rmse, test_mape, fc_rmse_list, fc_mape_list = get_arima_accuracy(df,best_lambda = best_lambdas[ind],fcs_nr_list = nr_of_forecasts_list, max_p = 5, max_q= 5, which_model = "ARIMA")
    train_rmse_list.append(train_rmse)
    train_mape_list.append(train_mape)
    test_rmse_list.append(test_rmse)
    test_mape_list.append(test_mape)
    fc_rmse_list_list.append(fc_rmse_list)
    fc_mape_list_list.append(fc_mape_list)
results = save_cv("results/ARIMA_results.csv", train_rmse_list, train_mape_list, test_rmse_list, test_mape_list, fc_rmse_list_list, fc_mape_list_list)


## SARIMA

In [ ]:
train_rmse_list, train_mape_list, test_rmse_list, test_mape_list, fc_rmse_list_list, fc_mape_list_list = [], [], [], [], [], []
nr_of_forecasts = [1, 3, 5, 10]
log_transformed_list, best_lambdas = log_transform(dfs_list)
for ind, df in enumerate(log_transformed_list):
    print(df.name)
    train_rmse, train_mape, test_rmse, test_mape, fc_rmse_list, fc_mape_list = get_arima_accuracy(df,best_lambda = best_lambdas[0],fcs_nr_list= nr_of_forecasts, max_p = 5, max_q= 5, max_Q = 5, max_P = 5, which_model = 'SARIMA')
    train_rmse_list.append(train_rmse)
    train_mape_list.append(train_mape)
    test_rmse_list.append(test_rmse)
    test_mape_list.append(test_mape)
    fc_rmse_list_list.append(fc_rmse_list)
    fc_mape_list_list.append(fc_mape_list)
results = save_cv("results/SARIMA_results.csv", train_rmse_list, train_mape_list, test_rmse_list, test_mape_list, fc_rmse_list_list, fc_mape_list_list)

   
